In [9]:
import splitfolders
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import ann

from typing import Any, Tuple
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import os

## Разделение данных

In [4]:
splitfolders.ratio('dataset', 'data_set_for_lerning',ratio=(0.8, 0.1, 0.1), seed=13, group_prefix=None)

Copying files: 2270 files [00:04, 556.20 files/s]


##  Модель сверточной нейронной сети

In [7]:
class CNN(nn.Module):
    def __init__(self) -> None:
        super(CNN, self).__init__()

        self.conv_1 = nn.Conv2d(3, 16, kernel_size=3, padding=0, stride=2)
        self.conv_2 = nn.Conv2d(16, 32, kernel_size=3, padding=0, stride=2)
        self.conv_3 = nn.Conv2d(32, 64, kernel_size=3, padding=0, stride=2)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.max_pool = nn.MaxPool2d(2)

        self.fc_1 = nn.Linear(576, 10)
        self.fc_2 = nn.Linear(10, 1)

    def forward(self, x: torch.tensor) -> torch.tensor:
        output = self.relu(self.conv_1(x))
        output = self.max_pool(output)
        output = self.relu(self.conv_2(output))
        output = self.max_pool(output)
        output = self.relu(self.conv_3(output))
        output = self.max_pool(output)

        
        output = torch.nn.Flatten()(output)
        output = self.relu(self.fc_1(output))
        output = torch.nn.Sigmoid()(self.fc_2(output))
        return output

## Предобработка данных

In [ ]:
class CustomImageDataset(Dataset):
  def __init__(self, path_to_annotation_file: str, transform: Any = None, target_transform: Any = None) -> None:
      self.path_to_annotation_file = path_to_annotation_file
      self.dataset_info = pd.read_csv(path_to_annotation_file, delimiter=';')
      self.transform = transform
      self.target_transform = target_transform

  def __len__(self) -> int:
      return len(self.dataset_info)

  def __getitem__(self, index: int) -> Tuple[torch.tensor, int]:
      path_to_image = self.dataset_info.iloc[index, 0]
      image = cv2.cvtColor(cv2.imread(path_to_image), cv2.COLOR_BGR2RGB)
      label = self.dataset_info.iloc[index, 1]

      if self.transform:
          image = self.transform(image)
      if self.target_transform:
          label = self.target_Transform(label)

      return image, label